# 기상청 단기예보 , 초단기실황 데이터 분석 (18.01~18.12)
### 전처리과정 거친 데이터 사용해서 분석

In [179]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import time

###  데이터 불러오기

In [182]:
s_real= pd.read_csv('D:/INC/Desktop/공모전/KB 국민카드/기상데이터/s_real.csv' ,index_col='new_date')

In [183]:
s_forecast= pd.read_csv('D:/INC/Desktop/공모전/KB 국민카드/기상데이터/s_forecast.csv')

## 1) Naive 분석 ( 전체 데이터 중, '맑은'  시간대 비율)

In [184]:
whole = len(s_real)*len(s_real.columns)

rain_snow = sum(sum(s_real.values==1))
good = sum(sum(s_real.values==0))

good_ratio = good/whole*100 

rain_snow_rati0 = (rain_snow)/whole*100


In [128]:
s_real

,s_57_125,s_57_126,s_57_127,s_58_124,s_58_125,s_58_126,s_58_127,s_59_124,s_59_125,s_59_126,...,s_61_128,s_61_129,s_62_125,s_62_126,s_62_127,s_62_128,s_62_129,s_63_125,s_63_126,s_63_127
new_date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2) forecast = 4 (4시간 뒤)에 대한 예측률

### 예측값과 발표시간 값을 datetime 형식으로 더해줌으로써, forecast_date 형성

In [186]:
s_forecast_4= s_forecast[s_forecast['forecast'] == 4 ]

s_forecast_4['forecast_date'] = dt.timedelta

for i in range(0,len(s_forecast_4)):
    s_forecast_4['forecast_date'].iloc[i] = pd.to_datetime(s_forecast_4['new_date'].iloc[i]) + dt.timedelta(hours = s_forecast_4['forecast'].iloc[i])
    
lis = list()
for i in range(len(s_forecast_4)):
    lis.append( str(s_forecast_4['forecast_date'].iloc[i]))
       

C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\INC\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### forecast_date를 index로 설정

In [135]:
s_forecast_4.set_index('forecast_date', drop=True ,append =False ,inplace=True)

### 필요없는 column 제거

In [136]:
s_forecast_4.drop(['new_date','forecast'], axis=1, inplace=True)

C:\Users\INC\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### 2019년을 예측하는 데이터는 제외

In [140]:
lis[2918]

'2019-01-01 00:00:00'

### 예측값과 실제값 비교하여, True값 result_4에 저장

In [141]:
result_4 = list()
for i in range(2918):
    result_4.append(sum(s_real.loc[lis[i]] == s_forecast_4.iloc[i]))

In [142]:
sum(result_4)

81746

In [143]:
whole = 30*2918

### 예보 성공률 계산

In [144]:
sum(result_4)/whole*100

93.38131140050263

### **예보 시간대에 따라, 예보 성공률 계산하는 과정은 위와 유사하므로 생략